# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [6]:
data.shape

(68584, 7)

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [38]:
grouping = data.groupby(['CustomerID', 'ProductName'], as_index=False)['Quantity'].sum()
grouping

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
5,33,"Beef - Chuck, Boneless",1
6,33,Beef - Prime Rib Aaa,1
7,33,Beer - Original Organic Lager,1
8,33,Beer - Rickards Red,1
9,33,Black Currants,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [16]:
matrix = pd.pivot_table(data, values='Quantity', index=['CustomerID'], columns=['ProductName'], aggfunc=np.sum, fill_value=0)
matrix.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [20]:
similarity = squareform(pdist(matrix, 'euclidean'))
similarity

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [35]:
distances = pd.DataFrame(1/(1 + squareform(pdist(matrix, 'euclidean'))), 
                         index=matrix.T.columns, columns=matrix.T.columns)

#Top 5 most similar customers to Customer ID 33
distances.iloc[1:,0].sort_values(ascending=False).head()

CustomerID
3535    0.087047
264     0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [42]:
top5 = [3535,264,3317,2503,3305]
top5_purchases = grouping[grouping['CustomerID'].isin(top5)]
top5_purchases.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [72]:
agg_purtchases = top5_purchases.groupby('ProductName')['Quantity'].sum().reset_index()
agg_purtchases = agg_purtchases.sort_values(by='Quantity', ascending=False)
agg_purtchases.head()

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [143]:
matrix2 = matrix.T.reset_index()
matrix2

CustomerID,ProductName,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,...,0,25,0,0,0,0,0,0,0,0
1,"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
2,Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,...,25,0,0,0,0,0,0,0,25,0
3,Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
4,Apricots - Dried,1,0,0,0,1,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
5,Apricots - Halves,0,0,1,0,0,0,0,0,0,...,0,0,0,0,25,50,25,0,25,25
6,Apricots Fresh,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,25
7,Arizona - Green Tea,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,25,0,0,0,0
8,Artichokes - Jerusalem,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Assorted Desserts,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,25,0,0,25


In [144]:
matrix2 = matrix2.merge(agg_purtchases, left_on='ProductName', right_on='ProductName')
matrix2


,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
0,Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,...,25,0,0,0,0,0,0,0,0,1
1,Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,1
2,Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,...,0,0,0,25,25,25,0,25,0,1
3,Apricots - Dried,1,0,0,0,1,0,0,0,0,...,25,0,0,0,0,0,0,0,0,1
4,Apricots - Halves,0,0,1,0,0,0,0,0,0,...,0,0,0,25,50,25,0,25,25,1
5,Apricots Fresh,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,25,1
6,Bacardi Breezer - Tropical,0,1,1,0,0,0,3,0,1,...,0,0,0,0,25,0,0,0,0,2
7,Bagel - Plain,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,25,0,50,0,1
8,Banana - Leaves,0,0,1,0,1,1,0,0,0,...,25,0,0,0,0,0,0,0,25,1
9,Bananas,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,25,1


In [151]:
matrix2 = matrix2.sort_values(by='Quantity', ascending=False)
matrix2

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
33,Butter - Unsalted,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25,0,3
199,Wine - Blue Nun Qualitatswein,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,25,0,3
129,"Pepper - Black, Whole",1,1,0,0,0,0,0,0,0,...,25,0,0,0,0,0,0,0,0,3
203,Wine - Ej Gallo Sierra Valley,0,0,0,1,0,0,0,0,0,...,0,25,50,0,25,0,0,0,0,3
186,Towels - Paper / Kraft,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
168,Soup - Campbells Bean Medley,0,0,1,0,0,0,2,0,0,...,0,0,25,50,25,0,0,0,0,3
12,Bar - Granola Trail Mix Fruit Nut,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
127,"Peas - Pigeon, Dry",0,1,0,0,0,0,0,1,0,...,25,25,0,0,25,0,0,0,0,2
160,Sea Bass - Whole,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,25,50,2
19,Beef - Rib Eye Aaa,0,0,0,0,1,0,0,0,0,...,25,25,0,0,0,0,0,0,0,2


In [154]:
matrix2[['ProductName',33,'Quantity']].loc[matrix2[33]==0].head()

,ProductName,33,Quantity
33,Butter - Unsalted,0,3
199,Wine - Blue Nun Qualitatswein,0,3
203,Wine - Ej Gallo Sierra Valley,0,3
168,Soup - Campbells Bean Medley,0,3
12,Bar - Granola Trail Mix Fruit Nut,0,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.